In [102]:
import pandas as pd
train_df = pd.read_csv('./preprocessed_train.csv', encoding='latin1')
test_df = pd.read_csv('./preprocessed_test.csv', encoding='latin1')

In [103]:
import gzip
from collections import defaultdict
import random
import numpy as np
import math
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.metrics import accuracy_score
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import TreebankWordTokenizer
from wordcloud import WordCloud
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
dataset = train_df.values.tolist()
test = test_df.values.tolist()

In [105]:
dataset = [d[2:] for d in dataset if (type(d[2]) != float and type(d[3]) != float)]
test = [d[2:] for d in test]
train = dataset[:37040]
validation = dataset[37040:]

In [106]:
train[0]

['Neutral',
 'MeNyrbie PhilGahan Chrisitv httpstcoiFz9FAn2Pa and httpstcoxX6ghGFzCC and httpstcoI2NlzdxNo8']

In [107]:
'''
text = comp_df.corpus.values
wordcloud = WordCloud(max_words=500,background_color='white', stopwords=stop_words, colormap='rainbow',height=300)
wordcloud.generate(str(text))
fig = plt.figure()
fig.set_figheight(6)
fig.set_figwidth(10)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
'''

"\ntext = comp_df.corpus.values\nwordcloud = WordCloud(max_words=500,background_color='white', stopwords=stop_words, colormap='rainbow',height=300)\nwordcloud.generate(str(text))\nfig = plt.figure()\nfig.set_figheight(6)\nfig.set_figwidth(10)\nplt.imshow(wordcloud)\nplt.axis('off')\nplt.show()\n"

In [108]:
wordCount_base = defaultdict(int)
punctuation = set(string.punctuation)
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for d in train:
    r = ''.join([c for c in d[1].lower() if (c not in punctuation and c not in numbers)])
    for w in r.split():
        if w not in stop_words:
            wordCount_base[w] += 1
counts_base = [(wordCount_base[w], w) for w in wordCount_base]
counts_base.sort()
counts_base.reverse()
words_base = [x[1] for x in counts_base[:4000]]
wordId_base = dict(zip(words_base, range(len(words_base))))
wordSet_base = set(words_base)

In [59]:
def feature_base(datum):
    feat = [0]*len(words_base)
    r = ''.join([c for c in datum[1].lower() if (c not in punctuation and c not in numbers)])
    for w in r.split():
        if w in words_base:
            feat[wordId_base[w]] += 1
    feat.append(1)
    return feat

In [60]:
X_train_base = [feature_base(d) for d in train]
y_train_base = [d[0] for d in train]
X_validation_base = [feature_base(d) for d in validation]
y_validation_base = [d[0] for d in validation]
X_test_base = [feature_base(d) for d in test]
y_test_base = [d[0] for d in test]

In [61]:
#X_train_base[100:200]

In [109]:
def change_sen(sentiment):
    if sentiment == "Extremely Positive":
        return 'positive'
    elif sentiment == "Extremely Negative":
        return 'negative'
    elif sentiment == "Positive":
        return 'positive'
    elif sentiment == "Negative":
        return 'negative'
    else:
        return 'netural'
y_train_base = [change_sen(s) for s in y_train_base]
y_validation_base = [change_sen(s) for s in y_validation_base]
y_test_base = [change_sen(s) for s in y_test_base]

In [64]:
from sklearn.linear_model import LogisticRegression
Xlil_base = scipy.sparse.lil_matrix(X_train_base)
clf_base = LogisticRegression(C=1, solver='lbfgs', max_iter=10000, multi_class='auto')
clf_base.fit(Xlil_base, y_train_base)
prediction_base = clf_base.predict(X_test_base)
acc_base = accuracy_score(prediction_base, y_test_base)
print(f'Word counts model accuracy on test data is {acc_base}')
print(classification_report(y_test_base, prediction_base))

Word counts model accuracy on test data is 0.7835703001579779
              precision    recall  f1-score   support

    negative       0.82      0.78      0.80      1633
     netural       0.66      0.68      0.67       619
    positive       0.80      0.83      0.82      1546

    accuracy                           0.78      3798
   macro avg       0.76      0.76      0.76      3798
weighted avg       0.78      0.78      0.78      3798



In [13]:
'''
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
lr = LogisticRegression(solver='lbfgs', max_iter=10000, multi_class='auto')
clf = GridSearchCV(lr, parameters)
clf.fit(Xlil_base, y_train_base)
clf.cv_results_
'''

"\nfrom sklearn.model_selection import GridSearchCV\nparameters = {'C':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}\nlr = LogisticRegression(solver='lbfgs', max_iter=10000, multi_class='auto')\nclf = GridSearchCV(lr, parameters)\nclf.fit(Xlil_base, y_train_base)\nclf.cv_results_\n"

In [65]:
wordCounts3 = defaultdict(int)
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for d in train:
    r = d[1].lower()
    r = ''.join([c for c in r if (c not in punctuation and c not in numbers)])
    ws = r.split()
    ws = ws + [' '.join(x) for x in zip(ws[:-1], ws[1:])]    
    for w in ws:
      if w not in stop_words:
        wordCounts3[w] += 1
    

print(len(wordCounts3))

490365


In [66]:
counts3 = [(wordCounts3[w], w) for w in wordCounts3]
counts3.sort()
counts3.reverse()

counts3 = counts3

words3 = [x[1] for x in counts3]

wordId3 = dict(zip(words3, range(len(words3))))
wordSet3 = set(words3)
print(len(words3))

490365


In [67]:
def feature3(datum):
    feat = [0]*len(words3)
    r = ''.join([c for c in datum[1].lower() if (c not in punctuation and c not in numbers)])
    ws = r.split()
    ws = ws + [' '.join(x) for x in zip(ws[:-1], ws[1:])]
    for w in ws:
        if w in words3:
            feat[wordId3[w]] += 1
    feat.append(1) #offset
    return feat

In [68]:
X_train3 = [feature3(d) for d in train]
y_train3 = [d[0] for d in train]
X_validation3 = [feature3(d) for d in validation]
y_validation3 = [d[0] for d in validation]

KeyboardInterrupt: ignored

In [ ]:
Xlil3 = scipy.sparse.lil_matrix(X_train3)
clf = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=10000, C = 1)
clf.fit(Xlil3, y_train3)
pred = clf.predict(X_validation3)
print(accuracy_score(pred, y_validation3))

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', class_weight='balanced', C = 10)
clf.fit(Xlil3, y_train3)
prediction = clf.predict(X_validation3)
print(accuracy_score(prediction, y_validation3))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1, solver='lbfgs', max_iter=10000, multi_class='auto')
clf.fit(Xlil3, y_train3)
prediction = clf.predict(X_validation3)
acc = accuracy_score(prediction, y_validation3)
print(f'Baseline model accuracy on test data is {acc}')

In [ ]:
'''
from sklearn.svm import SVC
clf = SVC(kernel='linear', class_weight='balanced', C = 10)
clf.fit(Xlil_base, y_train_base)
prediction = clf.predict(X_validation_base)
print(accuracy_score(prediction, y_validation_base))
'''

TF-IDF

In [110]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
tokenizer = TreebankWordTokenizer()
numbers = {'0','1','2','3','4','5','6','7','8','9'}
for d in train:
    r = ''.join([c for c in d[1].lower() if not (c in punctuation or c in numbers)])
    d[1] = r
    d[1] = tokenizer.tokenize(d[1])
    d[1] = [token for token in d[1] if token not in stop_words]
    for w in d[1]:
        wordCount[w] += 1

In [111]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words_list = [x[1] for x in counts]
words = set(words_list)
for d in validation:
    d[1] = tokenizer.tokenize(d[1])
    d[1] = [token for token in d[1] if token not in stop_words]
for d in test:
  d[1] = tokenizer.tokenize(d[1])
  d[1] = [token for token in d[1] if token not in stop_words]

In [112]:
train_X = []
train_y = []
for d in train:
    t = d[1]
    train_X.append(' '.join(t))
    train_y.append(d[0])
validation_X = []
validation_y = []
for d in validation:
    t = d[1]
    tnew = []
    for w in t:
        if w in words:
            tnew.append(w)
    validation_X.append(' '.join(tnew))
    validation_y.append(d[0])
test_X = []
test_y = []
for d in test:
    t = d[1]
    tnew = []
    for w in t:
        if w in words:
            tnew.append(w)
    test_X.append(' '.join(tnew))
    test_y.append(d[0])
train_y = [change_sen(s) for s in train_y]
validation_y = [change_sen(s) for s in validation_y]
test_y = [change_sen(s) for s in test_y]

In [113]:
test[0]

['Extremely Negative',
 ['TRENDING',
  'New',
  'Yorkers',
  'encounter',
  'empty',
  'supermarket',
  'shelves',
  'pictured',
  'Wegmans',
  'Brooklyn',
  'soldout',
  'online',
  'grocers',
  'FoodKick',
  'MaxDelivery',
  'coronavirusfearing',
  'shoppers',
  'stock',
  'httpstcoGr76pcrLWh',
  'httpstcoivMKMsqdT1']]

In [114]:
vectorizer = TfidfVectorizer(ngram_range = (1,2), max_features=4000, stop_words='english', max_df=0.5)
vectorizer.fit(train_X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=4000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [115]:
train_vector = vectorizer.transform(train_X)
validation_vector = vectorizer.transform(validation_X)
test_vector = vectorizer.transform(test_X)

In [116]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='saga', penalty='l1', multi_class='auto', max_iter=10000, C = 1)
clf.fit(train_vector, train_y)
print(clf.score(validation_vector, validation_y))
prediction = clf.predict(test_vector)
print(f'TF-IDF model accuracy on test set is {clf.score(test_vector, test_y)}')
print(classification_report(test_y, prediction))

0.7419198055893074
TF-IDF model accuracy on test set is 0.7345971563981043
              precision    recall  f1-score   support

    negative       0.78      0.74      0.76      1633
     netural       0.55      0.69      0.61       619
    positive       0.79      0.75      0.77      1546

    accuracy                           0.73      3798
   macro avg       0.70      0.72      0.71      3798
weighted avg       0.74      0.73      0.74      3798



In [86]:
test_vector[5]

<1x6000 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
'''
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}
lr = LogisticRegression(solver='lbfgs', max_iter=10000, multi_class='auto')
clf = GridSearchCV(lr, parameters)
clf.fit(train_vector, train_y)
clf.cv_results_
'''

In [82]:
pdct = clf.predict(validation_vector)

In [83]:
print(pdct[:10])
print(validation_y[:10])

['negative' 'netural' 'netural' 'netural' 'positive' 'positive' 'netural'
 'positive' 'netural' 'positive']
['negative', 'netural', 'positive', 'netural', 'positive', 'positive', 'netural', 'positive', 'netural', 'positive']


In [ ]:
print(validation[:3])

In [ ]:
'''
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors = 1)
clf.fit(train_vector, train_y)
prediction = clf.predict(validation_vector)
print(accuracy_score(prediction, validation_y))
print(prediction[:10])
'''

In [26]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', class_weight='balanced', C = 0.1)
clf.fit(train_vector, train_y)
prediction = clf.predict(validation_vector)
print(accuracy_score(prediction, validation_y))
print(prediction[:10])

0.6383961117861482
['negative' 'netural' 'netural' 'netural' 'positive' 'positive' 'netural'
 'positive' 'netural' 'positive']


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf'), 'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc = SVC(class_weight='balanced')
clf = GridSearchCV(svc, parameters)
clf.fit(train_vector, train_y)
clf.cv_results_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(n_estimators = 500, random_state=0)
clf.fit(train_vector, train_y)
prediction = clf.predict(validation_vector)
print(accuracy_score(prediction, validation_y))

In [ ]:
print(prediction[:10])

In [ ]:
print(validation_y)

In [ ]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words_list = [x[1] for x in counts[:2000]]
words = set(words_list)
for d in validation:
    d[1] = tokenizer.tokenize(d[1])
    d[1] = [token for token in d[1] if token not in stop_words]
train_X = []
train_y = []
for d in train:
    t = d[1]
    train_X.append(' '.join(t))
    train_y.append(d[0])
validation_X = []
validation_y = []
for d in validation:
    t = d[1]
    tnew = []
    for w in t:
        if w in words:
            tnew.append(w)
    validation_X.append(' '.join(tnew))
    validation_y.append(d[0])
vectorizer = TfidfVectorizer()
vectorizer.fit(train_X+validation_X)
train_vector = vectorizer.transform(train_X)
validation_vector = vectorizer.transform(validation_X)
train_tfidf = pd.DataFrame(train_vector.todense(), columns = vectorizer.get_feature_names())
validation_tfidf = pd.DataFrame(validation_vector.todense(), columns = vectorizer.get_feature_names())

In [ ]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=10000, C = 1)
clf.fit(train_tfidf, train_y)
print(clf.score(validation_tfidf, validation_y))